# Google Maps Review Summarizer

This Python app automates the process of fetching and summarizing Google Maps reviews for any business or location.

## Overview
The app performs two main tasks:
1. **Scrape Reviews** – Uses a web scraping script to extract reviews directly from Google Maps.
2. **Summarize Content** – Leverages OpenAI's language models to generate concise, insightful summaries of the collected reviews and analyse the sentiments.

## Tech Stack
- **Python** – Core language
- **Playwright** – For scraping reviews
- **OpenAI API** – For natural language summarization
- **Jupyter Notebook** – For exploration, testing, and demonstration

### Credits
The web scraping logic is **inspired by [Antonello Zanini’s blog post](https://blog.apify.com/how-to-scrape-google-reviews/)** on building a Google Reviews scraper. Special thanks for the valuable insights on **structuring and automating the scraping workflow**, which greatly informed the development of this improved scraper.

This app, however, uses an **enhanced version of the scraper** that can scroll infinitely to load more reviews until it collects **at least 1,000 reviews**. If only a smaller number of reviews are available, the scraper stops scrolling earlier.

**Note:** This project is intended for educational and research purposes. Please ensure compliance with Google’s [Terms of Service](https://policies.google.com/terms) when scraping or using their data.


In [ ]:
#Activate a virtual environment
!source ../../../.venv/bin/activate 

#Make sure pip is available and up to date inside the venv
!python3 -m ensurepip --upgrade

#Verify that pip now points to the venv path (should end with /.venv/bin/pip)
!which pip3

#Install Playwright inside the venv
!pip3 install playwright

#Download the required browser binaries and dependencies
!python3 -m playwright install

# the following not not needed

# Install the dotenv package
# !pip3 install python-dotenv

# Install the openai package
# !pip3 install openai

zsh:source:1: no such file or directory: ../../../.venv/bin/activate
Looking in links: /var/folders/n9/bqzhr3vs6rsc6v_m5w8bz3b40000gn/T/tmphu_nxjjc
/Users/jaechoi/repos/llm_replyGenix/.venv/bin/pip3
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s  0:00:00
Using cached anyio-4.11.0-py3-none-any.whl (109 kB)
Using cached distro-1.9.0-py3-no

In [4]:
import asyncio
from playwright.async_api import async_playwright
from IPython.display import Markdown, display
import os
from dotenv import load_dotenv
from openai import OpenAI


In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [6]:
async def scroll_reviews_panel(page, max_scrolls=50, max_reviews=10):
    """
    Scrolls through the reviews panel to lazy load all reviews.
    
    Args:
        page: Playwright page object
        max_scrolls: Maximum number of scroll attempts to prevent infinite loops
    
    Returns:
        Number of reviews loaded
    """
    # Find the scrollable reviews container
    # Google Maps reviews are in a specific scrollable div
    scrollable_div = page.locator('div[role="main"] div[jslog$="mutable:true;"]').first
    
    previous_review_count = 0
    scroll_attempts = 0
    no_change_count = 0

    print("Starting to scroll and load reviews...")
    
    while scroll_attempts < max_scrolls:
        # Get current count of reviews
        review_elements = page.locator("div[data-review-id][jsaction]")
        current_review_count = await review_elements.count()
        
        #if we have loaded max_reviews, we will stop scrolling
        if current_review_count >= max_reviews:
            break

        print(f"Scroll attempt {scroll_attempts + 1}: Found {current_review_count} reviews")
        
        # Scroll to the bottom of the reviews panel
        await scrollable_div.evaluate("""
            (element) => {
                element.scrollTo(0, element.scrollHeight + 100);
            }
        """)
        
        # Wait for potential new content to load
        await asyncio.sleep(2)
        
        # Check if new reviews were loaded
        if current_review_count == previous_review_count:
            no_change_count += 1
            # If count hasn't changed for 3 consecutive scrolls, we've likely reached the end
            if no_change_count >= 3:
                print(f"No new reviews loaded after {no_change_count} attempts. Finished loading.")
                break
        else:
            no_change_count = 0
        
        previous_review_count = current_review_count
        scroll_attempts += 1
    
    final_count = await review_elements.count()
    print(f"Finished scrolling. Total reviews loaded: {final_count}")
    return final_count

In [13]:
async def scrape_google_reviews(url):
    # Where to store the scraped data
    reviews = []

    async with async_playwright() as p:
         # Initialize a new Playwright instance
        browser = await p.chromium.launch(
            headless=True  # Set to False if you want to see the browser in action
        )
        context = await browser.new_context()
        page = await context.new_page()

        # The URL of the Google Maps reviews page

        # Navigate to the target Google Maps page
        print("Navigating to Google Maps page...")
        await page.goto(url)

        # Wait for initial reviews to load
        print("Waiting for initial reviews to load...")
        review_html_elements = page.locator("div[data-review-id][jsaction]")
        await review_html_elements.first.wait_for(state="visible", timeout=10000)
        
        # Scroll through the reviews panel to lazy load all reviews
        total_reviews = await scroll_reviews_panel(page, max_scrolls=100)
        
        print(f"\nStarting to scrape {total_reviews} reviews...")

        # Get all review elements after scrolling
        review_html_elements = page.locator("div[data-review-id][jsaction]")
        all_reviews = await review_html_elements.all()
        
        # Iterate over the elements and scrape data from each of them
        for idx, review_html_element in enumerate(all_reviews, 1):
            try:
                # Scraping logic

                stars_element = review_html_element.locator("[aria-label*=\"star\"]")
                stars_label = await stars_element.get_attribute("aria-label")

                # Extract the review score from the stars label
                stars = None
                for i in range(1, 6):
                    if stars_label and str(i) in stars_label:
                        stars = i
                        break

                # Get the next sibling of the previous element with an XPath expression
                time_sibling = stars_element.locator("xpath=following-sibling::span")
                time = await time_sibling.text_content()

                # Select the "More" button and if it is present, click it
                more_element = review_html_element.locator("button[aria-label=\"See more\"]").first
                if await more_element.is_visible():
                    await more_element.click()
                    await asyncio.sleep(0.3)  # Brief wait for text expansion

                text_element = review_html_element.locator("div[tabindex=\"-1\"][id][lang]")
                text = await text_element.text_content()

                reviews.append(str(stars) + " Stars: \n" +"Reviewed On:" + time + "\n"+ text)
                
                if idx % 10 == 0:
                    print(f"Scraped {idx}/{total_reviews} reviews...")
                    
            except Exception as e:
                print(f"Error scraping review {idx}: {str(e)}")
                continue

        print(f"\nSuccessfully scraped {len(reviews)} reviews!")

        # Close the browser and release its resources
        await browser.close()

        return "\n".join(reviews)

In [ ]:
test_url = "https://www.google.com/maps/place/Costco+Wholesale/@33.0355379,-96.8624758,14.29z/data=!4m8!3m7!1s0x864c23810890f0df:0xf9f33301a96d2ff4!8m2!3d33.0238859!4d-96.8314151!9m1!1b1!16s%2Fg%2F1tj58551?entry=ttu&g_ep=EgoyMDI1MTEwOS4wIKXMDSoASAFQAw%3D%3D"

In [ ]:
scrape_google_reviews(test_url)

In [ ]:
system_prompt = """
You are an expert assistnat that respond to google reviews,
and provide a detailed respond to given reviews.
Respond in markdonw. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [ ]:
# Define our user prompt

user_prompt_prefix = """
Here are the reviews of a google map location/business.
Provide a detailed respond of the reviews and the sentiment of the reviews.
"""

In [9]:

def prepare_message(reviews):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + reviews}
    ]

In [ ]:
async def respond_to_reviews(url):
    openai = OpenAI()
    reviews = await scrape_google_reviews(url)
    response = openai.chat.completions.create(
        model = "gpt-5-nano",
        messages = prepare_message(reviews)
    )
    return response.choices[0].message.content

In [ ]:
async def display_responses(url):
    responds = await respond_to_reviews(url)
    display(Markdown(responds))

In [17]:
url = "https://www.google.com/maps/place/Costco+Wholesale/@33.0355379,-96.8624758,14.29z/data=!4m8!3m7!1s0x864c23810890f0df:0xf9f33301a96d2ff4!8m2!3d33.0238859!4d-96.8314151!9m1!1b1!16s%2Fg%2F1tj58551?entry=ttu&g_ep=EgoyMDI1MTEwOS4wIKXMDSoASAFQAw%3D%3D"
await display_summary(url)


Navigating to Google Maps page...
Waiting for initial reviews to load...
Starting to scroll and load reviews...
Scroll attempt 1: Found 8 reviews
Finished scrolling. Total reviews loaded: 18

Starting to scrape 18 reviews...
Scraped 10/18 reviews...

Successfully scraped 18 reviews!


**Summary of reviews and overall sentiment**

- Overall tone: The collection is broadly positive, with a strong leaning toward 5-star experiences. There are a few negative reviews (1–2 stars) that highlight specific issues, mainly around customer service interactions, checkout experience, and a couple of service bottlenecks.

- Positive themes (recurrent across many reviews):
  - Good value and prices, especially for items like hearing aids, rotisserie chicken, and bakery items.
  - Helpful, friendly staff in several departments (e.g., Maddie in service, Raven and Rico in Optical, Maya in the store).
  - Clean, organized store environment and a generally good shopping experience.
  - Positive experiences with specific services (eye wear adjustments, cake design, car keys support, gas station convenience).
  - Consistent praise for staff going above and beyond and for quick, efficient checkout in many instances.

- Negative themes (less frequent, but notable):
  - Rude or dismissive behavior from a small number of associates, particularly at checkout.
  - Aggressive membership enforcement and sometimes unnecessary gatekeeping (e.g., being told to move or not allowed to certain areas).
  - Frustration with self-checkout changes and the perception that service quality is diminishing as a result.
  - Isolated complaints about Tire Center experiences and heat/discomfort while waiting, plus perceived bias or discrimination.
  - A few operational/booking notes (gas station construction, large weekend crowds, occasional delays).

- Notable reviews to watch for sentiment:
  - 5-star: Maddie’s service, Raven & Rico in Optical, Martha in bakery, and the hearing aid pricing/adjustment experiences.
  - 4-star: A long-standing, generally positive Costco experience with some friction due to staff interactions.
  - 1–2 star: Specific incidents of rude staff, discriminatory-feeling treatment, or dissatisfaction with checkout/self-checkout structure and customer service.

- Quick takeaway:
  - If you value Costco for price, breadth of products, and many staff-driven positive experiences, this location tends to deliver.
  - There are recurring cautions about occasional poor interactions with some staff, stricter membership enforcement, and some process changes that affect the checkout/assistance experience.

If you want, I can categorize the reviews by theme (pricing, staff behavior, checkout experience, services like Optical/Hearing Aids, etc.) or provide a one-sentence sentiment score per review.